In [ ]:
from time import sleep
from json import dumps
from kafka3 import KafkaProducer
import csv
import datetime as dt

In [ ]:
# hostip = "192.168.1.87" 
# hostip = "10.192.40.238"
hostip = "192.168.0.88"
topic = 'ProducerA'
file_path = 'A2/data/camera_event_A.csv'


def publish_message(producer_instance, topic_name, key, data):
    try:
        key_bytes = bytes(key, encoding='utf-8')
        value_bytes = bytes(data, encoding='utf-8')
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        print('Message published successfully. ' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        

In [ ]:
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=[f'{hostip}:9092'],
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer

In [13]:
if __name__ == '__main__':
    print('Publishing records..')
    producer = connect_kafka_producer()

    if producer is None:
        print('Failed to connect to Kafka. Exiting..')
        exit(1)

    current_batch = None
    batch_events = []

    with open(file_path, mode='r') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            json_row = dumps(row)
            if current_batch is None:
                current_batch = row['batch_id']

            if current_batch != row['batch_id']:
                for event in batch_events:
                    message = {
                        'event_id': event['event_id'],
                        "car_plate": event['car_plate'],
                        "camera_id": event['camera_id'],
                        "timestamp": event['timestamp'],
                        "speed_reading": float(event['speed_reading']),
                        "producer": "A"
                    }
                    publish_message(producer, topic, event['event_id'], dumps(message))
                producer.flush()
                print(f'Batch {current_batch} published successfully.')
                sleep(5)

                current_batch = row['batch_id']
                batch_events = []

            batch_events.append(row)

        if batch_events:
            for event in batch_events:
                message = {
                    'event_id': event['event_id'],
                    "car_plate": event['car_plate'],
                    "camera_id": event['camera_id'],
                    "timestamp": event['timestamp'],
                    "speed_reading": float(event['speed_reading']),
                    "producer": "A"
                }
                publish_message(producer, topic, event['event_id'], dumps(message))
            producer.flush()
            print(f'Final batch {current_batch} published successfully.')

    print('All messages published successfully.')

Publishing records..
Message published successfully. {"event_id": "d40c586c-5be6-4743-a1e3-2269d9edaa72", "car_plate": "KRN 7", "camera_id": "1", "timestamp": "2024-01-01T08:00:04", "speed_reading": 77.2, "producer": "A"}
Message published successfully. {"event_id": "85c08e3c-a0b5-45d8-a70c-df8f9a6d5829", "car_plate": "ICE 8", "camera_id": "1", "timestamp": "2024-01-01T08:00:05", "speed_reading": 103.7, "producer": "A"}
Message published successfully. {"event_id": "f5834b79-771b-4931-8da2-a5ad7f4ccd02", "car_plate": "QE 1820", "camera_id": "1", "timestamp": "2024-01-01T08:00:03", "speed_reading": 67.4, "producer": "A"}
Message published successfully. {"event_id": "d0e547bb-c4a7-4750-b7b4-8076e9b47f4f", "car_plate": "CJW 924", "camera_id": "1", "timestamp": "2024-01-01T08:00:01", "speed_reading": 148.3, "producer": "A"}
Message published successfully. {"event_id": "f3162606-1b2e-407f-951d-61d14c0a7b09", "car_plate": "CJP 278", "camera_id": "1", "timestamp": "2024-01-01T08:00:02", "speed

KeyboardInterrupt: 